In [1]:
import fdrot
import matplotlib.pyplot as plt
import numpy as np
from os import path

In [2]:
! conda install -n fdrot_dev -y numba

Solving environment: done

## Package Plan ##

  environment location: /home/ordyna35/anaconda3/envs/fdrot_dev

  added / updated specs: 
    - numba


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    numba-0.42.0               |   py36h962f231_0         2.7 MB
    llvmlite-0.27.0            |   py36hd408876_0        16.2 MB
    ------------------------------------------------------------
                                           Total:        18.9 MB

The following NEW packages will be INSTALLED:

    llvmlite: 0.27.0-py36hd408876_0
    numba:    0.42.0-py36h962f231_0


numba-0.42.0         | 2.7 MB    | ##################################### | 100% 
llvmlite-0.27.0      | 16.2 MB   | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [2]:
shape = (500, 2, 250)
input_rand_Bx = np.random.rand(*shape)
input_rand_Bz = np.random.rand(*shape)
input_rand_By = np.random.rand(*shape)
input_rand_n_e = np.random.rand(*shape)

In [4]:
# ! mkdir data_3d_testing
#! rm data_3d_testing/*
! ls data_3d_testing/

Bx1.npy  By1.npy  Bz1.npy  n_e1.npy


In [4]:
# fdrot.sim_data.__file__

In [3]:
for field, arr in [('Bx', input_rand_Bx),
             ('Bz', input_rand_Bz),
             ('By', input_rand_By),
             ('n_e', input_rand_n_e)]:
    np.save(path.join('data_3d_testing/', field + '1'), arr)

In [4]:
dt = 1
grid = (1,1,1)

In [5]:
def open_fkt(path:str, *args):
    return np.load(path)

In [6]:
files = {}
for field in ['Bx', 'Bz', 'By', 'n_e']:
    files[field] = fdrot.sim_data.UniversalSingle(
        path='data_3d_testing/',
        data_stored=field,
        single_time_step=dt,
        grid=grid,
        name_front=field,
        name_end='.npy',
        sim_box_shape=shape,
        export_func=open_fkt,
        axis_map=('z', 'y', 'x')
    )

In [7]:

sequence = fdrot.sim_sequence.seq_cells(start=0, end=500,
                             inc_time=1,
                             iter_step=1,
                             pulse_length_cells=1,
                             files=files, propagation_axis='z')

In [8]:
rotated = sequence.rotation_3d_perp(np.ones(1, dtype=np.float64), wavelength=1, second_axis_output='x')

In [9]:
fdrot.kernel3d.kernel3d.parallel_diagnostics(level=4)

 
 Parallel Accelerator Optimizing:  Function kernel3d, /net/fileu/user/ordyna35/F
araday_Rotation/faraday_rotation/calc_rotation/fdrot/kernel3d.py (5)  


Parallel loop listing for  Function kernel3d, /net/fileu/user/ordyna35/Faraday_Rotation/faraday_rotation/calc_rotation/fdrot/kernel3d.py (5) 
-------------------------------------------------------------------------------------------------------------------|loop #ID
@njit(parallel=True, cache=False)                                                                                  | 
def kernel3d(pulse: np.ndarray,                                                                                    | 
             input_arr: np.ndarray,                                                                                | 
             output: np.ndarray,                                                                                   | 
             global_start: int, global_stop: int,                                                        

In [14]:
Bx = np.load('data_3d_testing/Bx1.npy')
Bz = np.load('data_3d_testing/Bz1.npy')
n_e = np.load('data_3d_testing/n_e1.npy')

In [15]:
rotated.shape

(5, 8)

In [16]:
th = np.sum(Bz[0:8,:,:] * n_e[0:8,:,:], axis=0) * sequence.integration_factor(1) 

In [17]:
np.isclose(rotated, th, atol=1e-30)

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])

In [18]:
th

array([[4.77525829e-13, 4.03452204e-13, 4.87029085e-13, 4.76191276e-13,
        4.26014142e-13, 4.43367149e-13, 5.19621664e-13, 5.91361933e-13],
       [5.06430216e-13, 5.16437207e-13, 5.03935559e-13, 5.48231558e-13,
        7.61709907e-13, 6.94905576e-13, 7.18721691e-13, 7.48403582e-13],
       [4.35359728e-13, 3.58043112e-13, 3.29536378e-13, 4.24016578e-13,
        5.66278784e-13, 3.35168484e-13, 5.10519806e-13, 4.51692609e-13],
       [1.02134313e-13, 7.34014441e-13, 2.60845153e-13, 5.52255481e-13,
        4.95211941e-13, 5.08918013e-13, 3.75081962e-13, 4.72275773e-13],
       [3.22634984e-13, 5.48390130e-13, 3.96182690e-13, 6.47344667e-13,
        2.95472486e-13, 7.44730616e-13, 4.54036972e-13, 7.73418559e-13]])

In [19]:
rotated

array([[4.77525829e-13, 4.03452204e-13, 4.87029085e-13, 4.76191276e-13,
        4.26014142e-13, 4.43367149e-13, 5.19621664e-13, 5.91361933e-13],
       [5.06430216e-13, 5.16437207e-13, 5.03935559e-13, 5.48231558e-13,
        7.61709907e-13, 6.94905576e-13, 7.18721691e-13, 7.48403582e-13],
       [4.35359728e-13, 3.58043112e-13, 3.29536378e-13, 4.24016578e-13,
        5.66278784e-13, 3.35168484e-13, 5.10519806e-13, 4.51692609e-13],
       [1.02134313e-13, 7.34014441e-13, 2.60845153e-13, 5.52255481e-13,
        4.95211941e-13, 5.08918013e-13, 3.75081962e-13, 4.72275773e-13],
       [3.22634984e-13, 5.48390130e-13, 3.96182690e-13, 6.47344667e-13,
        2.95472486e-13, 7.44730616e-13, 4.54036972e-13, 7.73418559e-13]])